In [28]:
import os, sys
import pandas
import glob
import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
plt.style.use('seaborn-deep')
today = datetime.date.today()

In [29]:
# paths 
roadmap_path = "/dors/capra_lab/users/fongsl/roadmap/stringent/"
result_path = "%sresults/" %roadmap_path
output_path = "/dors/capra_lab/users/fongsl/broadly_active_enhancers/baes_data/villar_ref_data/"
os.chdir(result_path)

In [30]:
# Villar referenced against all roadmap samples
vr = pandas.read_table("%svillar_x_any_roadmap.bed" % result_path)

vr.head()

,v-chr,v-start,v-end,IDs,E120,E127,E101,E034,E104,E014,...,E113,E074,E026,E020,E091,E106,E029,E063,sum,90_percentile
0,chr20,270226,272388,hsaH3K27Ac23366,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,chr20,289482,291606,hsaH3K27Ac23368,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,chr20,342933,346519,hsaH3K27Ac23371,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,chr20,346432,347763,hsaH3K27Ac23372,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
4,chr20,350157,354996,hsaH3K27Ac23373,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,16.0,0.0


In [31]:
#'vsal' = 'Villar Species Activity/ aLignment' dataframe
vsal = pandas.read_table("/dors/capra_lab/projects/enhancer_uniqueness/data2/villar15/conservation/Hsap_Enhancers_conservationInOthers.bed", sep = '\t')

hq_species =['Mmul', 'Cjac', 'Mmus', 'Rnor', 'Ocun', 'Btau', 'Sscr', 'Cfam', 'Fcat']

species = vsal.columns[4:22]

In [32]:
#Step 1- sum activity across species
# "-" = alignable, not active

vsal["aln_inact"] = (vsal[species] == '-').sum(axis =1)
vsal["hq_aln_inact"] = (vsal[hq_species] == '-').sum(axis =1)

vsal[species] = vsal[species].replace({'H3': 1}, regex = True) # replace all the active enhancers with '1' to sum 
vsal["act_sp"] = (vsal[species] == 1).sum(axis=1)
vsal["hq_act_sp"] = (vsal[hq_species] ==1).sum(axis = 1)

In [33]:
#Step 2- sum alignable across species

vsal["aln_sp"] = vsal["aln_inact"] + vsal["act_sp"]
vsal["hq_aln_sp"]= vsal["hq_aln_inact"] + vsal["hq_act_sp"]

In [34]:
vsal.head()

,#Chrom,Start,End,IDs,Fcat,Csab,Mdom,Ddel,Mbid,Cpor,...,Tbel,Shar,Mmul,Mfur,aln_inact,hq_aln_inact,act_sp,hq_act_sp,aln_sp,hq_aln_sp
0,chr1,753206,753857,hsaH3K27Ac2,NaN,NaN,NaN,-,-,NaN,...,NaN,NaN,NaN,NaN,3,0,0,0,3,0
1,chr1,764441,766297,hsaH3K27Ac4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,chr1,839844,840941,hsaH3K27Ac5,-,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,1,0,0,1,1
3,chr1,859207,860479,hsaH3K27Ac6,1,NaN,NaN,1,-,1,...,NaN,NaN,NaN,-,3,0,3,1,6,1
4,chr1,877368,878855,hsaH3K27Ac7,-,NaN,NaN,NaN,NaN,-,...,NaN,-,NaN,-,4,1,0,0,4,1


In [35]:
vrs = pandas.merge(vr, vsal, how = "left", on= "IDs")
vrs.sort_values(by = "sum")
vrs.to_csv("%svillar_x_roadmap_x_species_overlap.csv" %output_path, sep = '\t', index = False, header = True)

In [36]:
# make a short dataframe for analysis

short_vrs = vrs[["v-chr", "v-start", "v-end","IDs", "sum","90_percentile", "act_sp", "aln_sp", "hq_act_sp", "hq_aln_sp", "aln_inact", "hq_aln_inact"]]
short_vrs= short_vrs.sort_values(by = "sum")
short_vrs["length"] = short_vrs["v-end"].subtract(short_vrs["v-start"])

short_vrs = short_vrs.fillna(0)
short_vrs.to_csv("%sshort_villar_x_roadmap_x_species_overlap.csv" %output_path, sep = '\t', index = False, header = True)

short_vrs_90 = short_vrs.loc[short_vrs["90_percentile"]==1]

In [37]:
############ LENGTH OF BROADLY ACTIVE ENHANCERS ########

# See vrs_boxplot_length.py

In [38]:
############ HUMAN-SPECIFIC/ SPECIES CONSERVED FOR GREAT ANNOTATION ####### 
comp = ["act_sp", "aln_sp", "hq_act_sp", "hq_aln_sp"]
for item in comp:
    
    hspec = short_vrs_90[["v-chr", "v-start", "v-end"]].loc[short_vrs_90['%s' %item] == 0]
    hspec.to_csv("%shspec_vrs_90_%s.bed" %(output_path, item), sep = '\t', index = False, header = False)
    
    consv = short_vrs_90[["v-chr", "v-start", "v-end"]].loc[short_vrs_90['%s'%item] > 0]
    consv.to_csv("%sconsv_vrs_90_%s.bed" %(output_path, item), sep = '\t', index = False, header = False)
   
    consv_10 = short_vrs_90[["v-chr", "v-start", "v-end"]].loc[short_vrs_90['%s'%item] >= 10]
    consv_10.to_csv("%sconsv_10_vrs_90_%s.bed" %(output_path,item), sep = '\t', index = False, header = False)
